In [0]:
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_colwidth', None)


In [0]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', names=['target', 'id', 'date', 'flag', 'user', 'text'])

In [0]:
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.chunk import conlltags2tree, tree2conlltags
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')

# Function to extract named entities from text
def extract_entities(text):
    words = word_tokenize(text)
    tagged = pos_tag(words)
    entities = ne_chunk(tagged)
    return entities

# Apply the function to the text column in your DataFrame
df['named_entities'] = df['text'].apply(extract_entities)
